# 基于Transformers的命名实体识别

## Step1 导入相关包

In [3]:
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

## Step2 加载数据集

In [4]:
# 如果可以联网，直接使用load_dataset进行加载
#ner_datasets = load_dataset("peoples_daily_ner", cache_dir="./data")
# 如果无法联网，则使用下面的方式加载数据集
#ner_datasets = DatasetDict.load_from_disk("ner_data")

from datasets import DatasetDict
ner_datasets = load_dataset("peoples_daily_ner", cache_dir="data", trust_remote_code=True)
ner_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [5]:
'''对应关系：
5 = B-LOC（地名开始）         6 = I-LOC（地名继续）
识别的实体：
"厦门"：[B-LOC, I-LOC]         "金门"：[B-LOC, I-LOC]
'''
ner_datasets["train"][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [6]:
ner_datasets["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [7]:
label_list = ner_datasets["train"].features["ner_tags"].feature.names
"""
B-：Begin，表示实体的开始,I-：Inside，表示实体的内部/继续,O：Outside，表示非实体
O——非实体——"的"、"在"、"和"
B-PE——人名开始——"张"（在"张三"中）
"""
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

## Step3 数据集预处理

In [8]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

In [9]:
tokenizer(ner_datasets["train"][0]["tokens"], is_split_into_words=True)
# 对ner_datasets文件夹里的训练集里的第1个样本，提取该样本的"tokens"字段，这通常是一个已经分词好的列表，告诉tokenizer输入已经是分词好的单词列表
'''
101 = 开始符号（就像"开始阅读"）
102 = 结束符号（就像"阅读结束"）

第一个句子的所有token对应0，第二个句子的所有token对应1
特殊标记[CLS]和[SEP]也会被赋予相应的句子ID。通常[CLS]和第一个句子后的[SEP]标记为0，第二个句子的token和最后的[SEP]标记为1
    假设有两个句子["今天天气很好", "我们出去玩"]，经过tokenizer后，可能变成：
    tokens: [CLS] 今 天 天 气 很 好 [SEP] 我 们 出 去 玩 [SEP]
    token_type_ids: 0 0 0 0 0 0 0 0 1 1 1 1 1 1

当一批(batch)中的句子长度不一致时需要填充，填充后：所有句子变成相同长度。值：1=需要关注，0=忽略
    '''

'\n101 = 开始符号（就像"开始阅读"）\n102 = 结束符号（就像"阅读结束"）\n\n第一个句子的所有token对应0，第二个句子的所有token对应1\n特殊标记[CLS]和[SEP]也会被赋予相应的句子ID。通常[CLS]和第一个句子后的[SEP]标记为0，第二个句子的token和最后的[SEP]标记为1\n    假设有两个句子["今天天气很好", "我们出去玩"]，经过tokenizer后，可能变成：\n    tokens: [CLS] 今 天 天 气 很 好 [SEP] 我 们 出 去 玩 [SEP]\n    token_type_ids: 0 0 0 0 0 0 0 0 1 1 1 1 1 1\n\n当一批(batch)中的句子长度不一致时需要填充，填充后：所有句子变成相同长度。值：1=需要关注，0=忽略\n    '

In [10]:
res = tokenizer("interesting word")
#BERT按照子词分的，所以interesting word被分成了几个子词（具体怎么分不需要知道），所以内部是5个数字
res


{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [11]:
res.word_ids()#返回每个token对应原始文本中的单词索引，说明前四个数字是interesting,后一个是word

[None, 0, 0, 0, 0, 1, None]

In [12]:
# 借助word_ids 实现标签映射
def process_function(examples):
    tokenized_exmaples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    '对example中的每一个名为"token"的字段进行分词，设置最大序列长度128，truncation=True如果超过128则截断，is_split_into_words=True告诉程序，输入（下载的数据集）已经分过词了，不要进一步分词'
    'BERT要用的三个输入值：input_ids，attention_mask，token_type_ids'

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_exmaples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)  #在计算损失时忽略
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_exmaples["labels"] = labels
    return tokenized_exmaples

In [13]:
tokenized_datasets = ner_datasets.map(process_function, batched=True)
'''
.map()
功能：对数据集中的每个样本应用指定的处理函数
返回：一个新的数据集，包含处理后的结果
'''
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [14]:
print(tokenized_datasets["train"][0])

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}


## Step4 创建模型

In [15]:
# 对于所有的非二分类任务，切记要指定num_labels，否则就会device错误
model = AutoModelForTokenClassification.from_pretrained("hfl/chinese-macbert-base", num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model.config.num_labels

7

## Step5 创建评估函数

In [17]:
# 这里方便加载，替换成了本地的加载方式，无需额外下载
seqeval = evaluate.load("seqeval_metric.py")
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [18]:
#这个函数接收模型预测结果，计算NER任务的F1分数。
import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    'pred 是模型输出的预测结果和真实标签,predictions: 模型的原始输出（通常是logits）,labels: 真实的标签序列'
    predictions = np.argmax(predictions, axis=-1)   #返回该行/列（axis=0，看每一列，axis=-1，从倒一行开始看）最大索引，
    # 在二维数组中，axis=-1和axis=1是一样的，只不过从倒数第一行开始看

    # 将id转换为原始的字符串类型的标签
    true_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    '''
    在PyTorch中，-100 是交叉熵损失函数的默认"忽略索引".当标签为-100时，该位置的损失不会被计算，梯度也不会回传（被忽略）。
    具体哪些位置会被标记为-100？
        [CLS] token：序列开始标记
        [SEP] token：序列结束标记
        Padding tokens：填充位置（如果序列长度不足max_length）
        子词的后续部分：一个单词被拆成多个子词时，只有第一个子词保留标签，后续子词标记为-100
    '''

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")

    return {
        "f1": result["overall_f1"]
    }
    

## Step6 配置训练参数

In [19]:
args = TrainingArguments(
    output_dir="models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=50,   #每过50个batch打印一次
    num_train_epochs=1
)

## Step7 创建训练器

In [20]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

## Step8 模型训练

In [19]:
trainer.train()

D:\Anaconda\envs\llm-project\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,F1
1,0.025000,0.020442,0.940093


TrainOutput(global_step=327, training_loss=0.06583504757020817, metrics={'train_runtime': 665.6072, 'train_samples_per_second': 31.347, 'train_steps_per_second': 0.491, 'total_flos': 1317626511207666.0, 'train_loss': 0.06583504757020817, 'epoch': 1.0})

In [20]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

{'eval_loss': 0.024249466136097908,
 'eval_f1': 0.9311757201369771,
 'eval_runtime': 66.0139,
 'eval_samples_per_second': 70.243,
 'eval_steps_per_second': 0.56,
 'epoch': 1.0}

## Step9 模型预测

In [21]:
from transformers import pipeline

In [22]:
# 使用pipeline进行推理，要指定id2label
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config

BertConfig {
  "_name_or_path": "hfl/chinese-macbert-base",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  

In [23]:
# 如果模型是基于GPU训练的，那么推理时要指定device
# 对于NER任务，可以指定aggregation_strategy为simple，得到具体的实体的结果，而不是token的结果
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, device=0, aggregation_strategy="simple")

In [24]:
res = ner_pipe("小明在北京上班")
res

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'PER',
  'score': 0.94918025,
  'word': '小 明',
  'start': 0,
  'end': 2},
 {'entity_group': 'LOC',
  'score': 0.9956497,
  'word': '北 京',
  'start': 3,
  'end': 5}]

In [25]:
# 根据start和end取实际的结果
ner_result = {}
x = "小明在北京上班"
for r in res:
    if r["entity_group"] not in ner_result:
        ner_result[r["entity_group"]] = []
    ner_result[r["entity_group"]].append(x[r["start"]: r["end"]])

ner_result

{'PER': ['小明'], 'LOC': ['北京']}

In [ ]:
"""
B-：Begin，表示实体的开始    I-：Inside，表示实体的内部/继续
O：Outside，表示非实体

O	     非实体	"的"、"在"、"和"
B-PER	人名开始	"张"（在"张三"中）
I-PER	人名继续	"三"（在"张三"中）
B-ORG	组织机构开始	"阿里"（在"阿里巴巴"中）
I-ORG	组织机构继续	"巴巴"（在"阿里巴巴"中）
B-LOC	地名开始	"北"（在"北京"中）
I-LOC	地名继续	"京"（在"北京"中）

例如：
张      三   在   北     京   的   阿        里   巴   巴   工
B-PER I-PER O   B-LOC I-LOC O   B-ORG I-ORG I-ORG I-ORG O
 作
 O
"""